In [ ]:
!wget https://www.dropbox.com/s/viz3bmc8cil4w1y/train_data.csv?dl=1
!wget https://www.dropbox.com/s/07wu5by7llczd36/test_data.csv?dl=1
!wget https://www.dropbox.com/s/meymqbp1wrl23q6/sample_solution.csv?dl=1
!pip install gensim

--2023-04-11 14:28:21--  https://www.dropbox.com/s/viz3bmc8cil4w1y/train_data.csv?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6025:18::a27d:4512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/viz3bmc8cil4w1y/train_data.csv [following]
--2023-04-11 14:28:22--  https://www.dropbox.com/s/dl/viz3bmc8cil4w1y/train_data.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6f1493f8cddc0f8d2cb7980b60.dl.dropboxusercontent.com/cd/0/get/B58es7qZCzsmvehxnQawYTBliX7qpfm-sr7PrH8VR_Hx-VwsWVx9nylKOjikpWVl36jtz3XaihE5aAJXo9fmVpm6TWlu7zmttxeeLeL32ItCHNOP_KR9mekmiJ4k1jxDnLB5Y-zy0kenYvGBu3QyUiHZJMUbMjauZ3mwTaN-k8cv8Q/file?dl=1# [following]
--2023-04-11 14:28:22--  https://uc6f1493f8cddc0f8d2cb7980b60.dl.dropboxusercontent.com/cd/0/get/B58es7qZCzsmvehxnQawYTBliX7qpfm-sr7PrH8VR_Hx-VwsWVx9nylKOjikpWVl36jtz

In [ ]:
!pip uninstall umap
!pip install umap-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82830 sha256=87bd1e38f09d8eef9a8591f76b0857db7ad5e3a90068ee2a305b18cb29fbc31e
  Stored in directory: /root/.cache/pip/wheels/f4/3e/1c/596d0a463d17475af648688443fa4846fef624d1390339e7e9
  Created wheel for pynndescent: filename=pynndescent-0.5.8-py3-none-any.whl size=55509 sha256=6ee62d7525a0df18e00373e0a33341809cb1597409a26287fed6c3d2905f6233
  Stored in directory: /root/.cache/pip/wheels/b9/89/cc/59ab91ef5b21dc2ab3635528d7d227f49dfc9169905dcb959d
Successfully built umap-learn pynndescent


**Other embeddings are available**

If you'd like to experiment with other embeddings, you can check out the full list of available ones [here](https://github.com/RaRe-Technologies/gensim-data)

In [ ]:
import gensim.downloader as api
word_vectors = api.load('glove-wiki-gigaword-100') 

[==================================================] 100.0% 128.1/128.1MB downloaded


**Playing with the word embeddings**

What's the most similar word to cat?

In [ ]:
word_vectors.most_similar('cat')

[('dog', 0.8798074722290039),
 ('rabbit', 0.7424427270889282),
 ('cats', 0.732300341129303),
 ('monkey', 0.7288709878921509),
 ('pet', 0.719014048576355),
 ('dogs', 0.7163872718811035),
 ('mouse', 0.6915250420570374),
 ('puppy', 0.6800068020820618),
 ('rat', 0.6641027331352234),
 ('spider', 0.6501135230064392)]

Now let's solve the famous analogy:

man:king as woman:?

We can solve this using vector operations:
king - man + woman = ?


In [ ]:
word_vectors.most_similar(positive=['king','woman'], negative=['man'])

[('queen', 0.7698540687561035),
 ('monarch', 0.6843381524085999),
 ('throne', 0.6755736470222473),
 ('daughter', 0.6594556570053101),
 ('princess', 0.6520534157752991),
 ('prince', 0.6517034769058228),
 ('elizabeth', 0.6464517712593079),
 ('mother', 0.631171703338623),
 ('emperor', 0.6106470823287964),
 ('wife', 0.6098655462265015)]

**Process the data**

In [ ]:
import pandas as pd
import numpy as np
train_data = pd.read_csv('train_data.csv?dl=1', encoding = 'latin-1')
test_data = pd.read_csv('test_data.csv?dl=1', encoding = 'latin-1')
train_data.iloc[0:5]

## Tokenize ##
from tensorflow.keras.preprocessing.text import Tokenizer
texts = train_data['Text']
test_sentences = test_data['Text']
labels = train_data['Label']

max_words = 10000
token = Tokenizer(max_words)
token.fit_on_texts(texts)
vocab_size = max_words + 1

sequences = token.texts_to_sequences(texts)
test_sequences = token.texts_to_sequences(test_sentences)

## Convert to sequences ##
from tensorflow.keras.preprocessing.sequence import pad_sequences
seq_len = 125
X = pad_sequences(sequences, maxlen=seq_len)
X_test = pad_sequences(test_sequences, maxlen=seq_len)

## Make y one-hot ##
y = np.zeros( (len(labels),len(np.unique(labels)) ) )
for l in np.unique(labels):
    pos_inds = np.where(labels == l)[0]
    y[pos_inds,l] = 1

num_classes = y.shape[1]


In [ ]:
embeddings_index = {}
vocab = token.word_index.keys()
for word in vocab:
    if word in word_vectors.index_to_key:
      coefs = np.asarray(word_vectors[word], dtype='float32')
      embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Found 56914 word vectors.


In [ ]:
word_index = token.word_index
embedding_matrix = np.zeros( (vocab_size , 100) )
for word, i in word_index.items():
    if i < vocab_size:
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
          # words not found in embedding index will be all-zeros.
          embedding_matrix[i] = embedding_vector

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Activation, Dropout, Embedding, Input, Dropout, LSTM, GRU

input = Input(shape=(seq_len,))
x = Embedding(input_dim = vocab_size , output_dim = 100, weights=[embedding_matrix], name='embedding_layer')(input)
x = LSTM(units = 128, name='document_embeddings')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(units=20, activation='softmax', name='output')(x)

model = Model(input,x)

In [ ]:
len(np.where(embedding_matrix.sum(axis=1) == 0)[0])

744

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X,y,epochs=10, validation_split=0.2, batch_size = 128)

Epoch 1/10
71/71 [==============================] - 23s 209ms/step - loss: 2.9476 - accuracy: 0.0849 - val_loss: 2.8256 - val_accuracy: 0.1675
Epoch 2/10
71/71 [==============================] - 12s 165ms/step - loss: 2.5523 - accuracy: 0.1680 - val_loss: 2.2389 - val_accuracy: 0.2430
Epoch 3/10
71/71 [==============================] - 10s 137ms/step - loss: 2.1498 - accuracy: 0.2563 - val_loss: 1.9946 - val_accuracy: 0.3230
Epoch 4/10
71/71 [==============================] - 6s 89ms/step - loss: 1.9626 - accuracy: 0.3119 - val_loss: 1.7052 - val_accuracy: 0.4176
Epoch 5/10
71/71 [==============================] - 6s 87ms/step - loss: 1.5889 - accuracy: 0.4208 - val_loss: 1.4431 - val_accuracy: 0.4861
Epoch 6/10
71/71 [==============================] - 5s 67ms/step - loss: 1.3271 - accuracy: 0.5118 - val_loss: 1.3177 - val_accuracy: 0.5347
Epoch 7/10
71/71 [==============================] - 4s 50ms/step - loss: 1.0976 - accuracy: 0.6005 - val_loss: 1.0912 - val_accuracy: 0.6310
Epoch 8

In [ ]:
embedding_model = Model(inputs=input, outputs=model.get_layer('document_embeddings').output)
doc_embeddings = embedding_model.predict(X)
print(doc_embeddings.shape)

354/354 [==============================] - 1s 3ms/step
(11314, 128)


In [ ]:
import umap.umap_ as umap
reducer = umap.UMAP()
doc_umap = reducer.fit_transform(doc_embeddings)

In [ ]:
import pandas as pd
import plotly.express as px

embedding_df = pd.DataFrame({
    'D1': doc_umap[:, 0],
    'D2': doc_umap[:, 1],
    'Label': labels.astype('str')
})

fig = px.scatter(embedding_df, x="D1", y="D2", color='Label', width=1000, height=750)
fig.show()

In [ ]:
test_preds = model.predict(X_test, batch_size=256)
submission_template = pd.read_csv('sample_solution.csv?dl=1')

for j in range(test_preds.shape[1]):
  submission_template.iloc[:,j+1] = test_preds[:,j]



30/30 [==============================] - 1s 10ms/step


In [ ]:
from google.colab import files
submission_template.to_csv('submission_pretrained.csv', index=False)
files.download('submission_pretrained.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 125)]             0         
                                                                 
 embedding_layer (Embedding)  (None, 125, 100)         1000100   
                                                                 
 document_embeddings (LSTM)  (None, 128)               117248    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 256)               33024     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 output (Dense)              (None, 20)                5140  